In [ ]:
%pylab inline
import scipy.interpolate as spi
import scipy.integrate as spint
import scipy.signal as spsig
import sys, os

sys.path.insert(0, os.path.abspath('../Rayleigh'))

import post_processing.rayleigh_diagnostics as rd
import post_processing.reference_tools as rt
import stelo.model_reader as mr

In [ ]:
g0 = 1.414870
length = 4e8
q0 = 3.795720e-4
gasconst = 8.314463e7
ref_pres = 4.644481e23
ref_density = 1.820940e6
ref_grav = 6.376489e8

In [ ]:
nr = 10000
r0 = 1
r1 = 3
radius = np.linspace(r0, r1, nr)

In [ ]:
def fg(r):
    return np.where((1 + 1./16. <= r) & (r <= 3 - 1./16.),
                    1.,
                    0.5 * (1. + np.sin(16 * np.pi * (r - 1./32.)))
                   )

In [ ]:
def eta1(r):
    return np.where((2. - 1./16. <= r) & (r <= 2. + 1./16.),
                    0.5 * (1. + np.sin(8. * np.pi * r)),
                    np.where(r > 2. + 1./16.,
                             1., 0.))

In [ ]:
def dlnpdlnrho(r):
    y0 = 5. / 3.
    y1 = 1.3
    return np.where((2. - 1./16. <= r) & (r <= 2. + 1./16.),
                    y0 + eta1(r) * (y1 - y0),
                    np.where(r > 2. + 1./16,
                             y1, y0))

In [ ]:
def eos(rho, T, mu):
    return gasconst * rho * T / mu

In [ ]:
grav = g0 * fg(radius) * radius**(-5./4.)

In [ ]:
def heating(r):
    return np.where(r < 1. + 1./8.,
                    q0 * np.sin(8. * np.pi * r),
                    0.)

In [ ]:
r, g, fv, rho, pres = np.loadtxt('setup-two-layers.in',
                                 skiprows=1, unpack=True)

In [ ]:
mu0 = 1.848
mu1 = 1.802
mu = (1. - fv)*mu0 + fv*mu1

In [ ]:
temp = pres * ref_pres / (gasconst * rho * ref_density) * mu
gamma = 5./3.
cp = gasconst * gamma / (gamma -1)

In [ ]:
plot(r,temp)

In [ ]:
density = rho * ref_density
temperature = temp
grav = g * ref_grav
buoy = density * grav / cp
nu = np.ones_like(r)
kappa = np.ones_like(r)
eta = np.ones_like(r)
hprofile = heating(r)
dsdr = (cp / (gamma * temperature) *
        (np.gradient(temperature, r) -
         (gamma - 1) * temperature * np.gradient(np.log(density), r))
       )

In [ ]:
temperature

In [ ]:
plot(r,dsdr)

In [ ]:
m = mr.model()
m.r = r * length
m.grav = grav
m.rho = density
m.T = temperature
m.P = pres * ref_pres
m.eos_mode = ('gas',)
m.Zbar = np.zeros_like(m.r)
m.Abar = np.ones_like(m.r) * mu

In [ ]:
plot(m.r, m.brunt(schwarzschild=False))
plot(m.r, m.brunt(schwarzschild=True))

In [ ]:
plot(m.hystat(rotation=False)[1000:-1000])

In [ ]:
my_ref = rt.equation_coefficients(r)

my_ref.density = density
my_ref.buoy = buoy
my_ref.buoy_fact = 1.0

# There are all normalized to 1.
# They can be adjusted with the corresponding factors.
my_ref.nu = nu
my_ref.kappa = kappa
my_ref.eta = eta

my_ref.visc_fact = 1e12
my_ref.diff_fact = 1e12
my_ref.resist_fact = 1e12

my_ref.temperature = temperature
my_ref.p_fact = 1.0
my_ref.ds_dr = dsdr
my_ref.lorentz_fact = 0.25/np.pi

my_ref.heating = hprofile
# This can be overridden to boost the luminosity.
my_ref.luminosity = 1.0

In [ ]:
file_write='cref_code_comp.dat'
my_ref.write(file_write)